# Comment gérer des variables catégorielles

Les données récoltées lors d’une enquête ne sont guère toutes sous forme numérique, et encore, même si elles le sont, elles peuvent malgré tout être catégorielles. Une variable *satisfaction* avec deux valeurs possibles *0* et *1* laisse bien penser qu’il existe deux catégories d’observations : les satisfaites et les insatisfaites. Une autre variable *gender* pourrait bien elle aussi être codée *0* pour les hommes et *1* pour les femmes.

L’enjeu de la prise en charge des variables catégorielles, encore plus prégnant dans le domaine du TAL, réside dans les phases de pré-traitement, par des moyens d’encodage, en vue de les injecter dans les algorithmes d’apprentissage.

## Qu’est-ce qu’une variable catégorielle ?

Autrement nommées qualitatives ou factorielles en statistiques, les variables catégorielles ne mesurent pas une quantité mais prennent comme valeurs une modalité.

### Identifier une variable catégorielle

Jusqu’à présent, les exemples ne recouraient qu’à des variables numériques, identifiées par la propriété `dtypes` d’un *data frame*. La même méthode peut être employée pour cibler les variables de type `object` :

In [ ]:
import numpy as np
import pandas as pd

df = pd.DataFrame({
    'gender': ['F', 'M', 'M', np.nan, np.nan],
    'height': [180, 172, 167, 187, 173]
})

df.dtypes

Pour accéder à la liste des catégories, triées par fréquence, il faut appeler la méthode `.value_counts()` :

In [ ]:
df["gender"].value_counts()

Et chaîner avec la méthode `.sort_index()` pour obtenir plutôt un tri sur les étiquettes :

In [ ]:
df["gender"].value_counts().sort_index()

### Isoler les variables catégorielles

Lors de la phase de pré-traitement des données, il est souvent judicieux de séparer les variables quantitatives des variables catégorielles afin de leur appliquer des transformations différentes. Une fonction `make_column_selector()` peut s’avérer utile dans le cas où les deux sont clairement discriminées par leur type :

In [ ]:
from sklearn.compose import make_column_selector as col_selector

# a selector for categorical variables
categorical_selector = col_selector(dtype_include=object)
# apply to the data frame
categorical_cols = categorical_selector(df)

# a new data frame with only categorical variables
data_categorical = df[categorical_cols]

## Traiter les données manquantes

L’étape de gestion des données manquantes est aussi cruciale que pour les variables quantitatives. Dans certains cas, il vous sera possible de la conduire après la phase d’encodage sous forme numérique ; dans d’autres, vous préférerez la résoudre tant que les catégories sont compréhensibles par un agent humain.

### Supprimer les données manquantes

La première stratégie consiste à supprimer purement et simplement les données manquantes, au risque de pénaliser les performances de votre programme. Tout dépend, toujours, de la quantité de données à disposition : une centaine d’observations sur des millions consignées dans le *dataset* ne devrait pas peser bien lourd là où une centaine sur un millier causerait un biais dommageable. À vous de définir la limite !

Pour la méthode, `.dropna()` :

In [ ]:
data_del_na = data_categorical.dropna()
data_del_na

### Adopter la catégorie la plus fréquente

Si votre option est de combler les données manquantes avec la modalité la plus représentée, prenant ainsi le risque d’introduire un biais de représentativité, la classe `SimpleImputer` est toute indiquée :

In [ ]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=pd.NA, strategy="most_frequent")
data_most_frequent = imputer.fit_transform(data_categorical)
data_most_frequent

### Remplacer par une valeur par défaut

La plupart du temps, vous souhaiterez assigner une valeur par défaut, soit de votre choix parmi la liste des autres modalités de la variable, soit une étiquette spécifique qui deviendra alors une catégorie à part entière. La méthode la plus simple est alors d’utiliser la méthode `.fillna()` :

In [ ]:
data_categorical = data_categorical.fillna("?")
data_categorical

### Établir une stratégie personnalisée

Si vous souhaitez gérer plus finement les données manquantes et les remplacer par les valeurs les plus probables, comme inférer le genre d’un individu en fonction de sa taille, vous préférerez alors mettre en place une stratégie elle-même basée sur un algorithme d’apprentissage. Ci-dessous un exemple simplifié et hautement défectueux :

In [ ]:
from sklearn.linear_model import LogisticRegression

target = "gender"
features = ["height"]

data = df[features + [target]]
data = data.dropna()

y = data[target]
X = data.drop(columns=target).values

model = LogisticRegression()
model.fit(X, y)

# work on a copy
df_copy = df.copy()

# a mask to select multiple items: only the pd.NA
mask = df_copy['gender'].isna()

# setting items: gender filled with predictions
# based on its corresponding height
df_copy.loc[mask, 'gender'] = model.predict(
    df_copy.loc[mask, 'height'].values.reshape(-1, 1)
)

df_copy

## Recoder les variables catégorielles

### Un encodage avec une hypothèse sur l’ordre des catégories

La méthode la plus simple pour recoder les variables catégorielles serait d’attribuer un nombre à chaque catégorie. C’est le rôle de la classe `OrdinalEncoder` du module `sklearn.preprocessing` :

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
data_categorical_encoded = encoder.fit_transform(data_categorical)
data_categorical_encoded

*Scikit-Learn* ne perd pas pour autant la trace des modalités d’origine. Elles sont accessibles via la propriété `categories_` :

In [ ]:
encoder.categories_

Le défaut majeur de ce type d’encodeur réside dans le fait qu’il utilise une stratégie lexicographique pour encoder les différentes modalités, en supposant que l’ordre (1, 2, 3…) est porteur de sens. Ce traitement peut introduire un biais dans les prédictions d’un modèle.

Dans notre exemple, les modalités, après avoir été triées par ordre alphabétique, revêtent les étiquettes suivantes :

|Modalité|Étiquette|
|:-:|:-:|
|?|0|
|F|1|
|M|2|

Pour remédier à ce traitement par défaut, il est possible de passer au constructeur un argument `categories` :

In [ ]:
encoder = OrdinalEncoder(categories=[['F', '?', 'M']])
data_categorical_encoded = encoder.fit_transform(data_categorical)
data_categorical_encoded

### Un vecteur *one-hot*

Pour remédier au défaut de la classe `OrdinalEncoder`, il existe un autre encodeur qui n’effectue aucune supposition préalable sur l’ordre des catégories : `HotOneEncoder`. Sa stratégie est un peu gourmande en espace :
1. Pour chaque observation, la modalité est encodée par un tableau ;
2. ce tableau est composé d’autant d’éléments que de modalités au total ;
3. les éléments adoptent deux valeurs possibles, `0` ou `1`, en fonction de leur position et de la modalité de l’observation.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# without argument 'sparse', matrix would not been displayed
encoder = OneHotEncoder(sparse=False)
data_categorical_encoded = encoder.fit_transform(data_categorical)
data_categorical_encoded

Le résultat de la transformation est une matrice creuse (peu de relations entre les données, ce qui est matérialisé par la sur-représentation de `0` dans la matrice). Si elle occupe beaucoup d’espace, elle n’est pas forcément gourmande en mémoire : elle ne stocke que l’emplacement des éléments différents de `0`.

Essayons de nous représenter les opérations réalisées par l’encodeur, en étant toutefois conscient qu’elles ne correspondent pas à la réalité de l’algorithme. À l’état originel, la variable *gender* peut être représentée ainsi :

|**gender**|
|:-:|
|F|
|M|
|M|
|?|
|?|

Les trois modalités possibles étant, une fois triées, `['?', 'F', 'M']`, elles sont comparées à la modalité présente :

|**gender**|
|:-:|
|F `['?', 'F', 'M']`|
|M `['?', 'F', 'M']`|
|M `['?', 'F', 'M']`|
|? `['?', 'F', 'M']`|
|? `['?', 'F', 'M']`|

À l’endroit du tableau où les modalités correspondent, l’encodeur place un `1` :

|**gender**|
|:-:|
|`['?', 1, 'M']`|
|`['?', 'F', 1]`|
|`['?', 'F', 1]`|
|`[1, 'F', 'M']`|
|`[1, 'F', 'M']`|

Et les autres sont fixées à `0` :

|**gender**|
|:-:|
|`[0, 1, 0]`|
|`[0, 0, 1]`|
|`[0, 0, 1]`|
|`[1, 0, 0]`|
|`[1, 0, 0]`|

### La gestion des catégories rares

Bien souvent, lorsque le nombre de modalités commence à devenir important, certaines n’apparaîtront que rarement. Il arrive alors que, au moment de l’étape où les données sont séparées entre les jeux d’entraînement et de test, elles ne soient pas contenues dans l’un ou l’autre. Pour éviter la levée d’erreurs au moment de la prédiction, les deux encodeurs disposent d’un paramètre `handle_unknown` :

In [ ]:
ordinal_enc = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
hot_enc = OneHotEncoder(handle_unknown="ignore")

## Comment choisir la bonne méthode pour encoder une variable catégorielle ?

La décision que vous prendrez reposera essentiellement sur le type de variable catégorielle (ordonnée ou non) et sur le modèle que vous choisirez en aval pour l’apprentissage de votre programme. Les modèles linéaires sont très fortement impactées par une hypothèse fondée sur l’ordre des catégories quand les modèles arborescents ne le sont pas du tout. On préférera donc `OneHotEncoder` pour les premiers et `OrdinalEncoder` pour les seconds.

Pour autant, il est toujours possible d’utiliser un `OrdinalEncoder` avec un modèle linéaire si l’on est bien certains que les variables explicatives sont ordonnées.

## Diriger des variables qualitatives et quantitatives dans un même pipeline

Il existe une classe qui permet d’exécuter l’ensemble des transformateurs avant d’envoyer les données au modèle à entraîner : `ColumnTransformer`.

Avant de l’exploiter, utilisons notre fonction `col_selector()` définie plus haut pour retenir les colonnes de type numérique par exclusion de celles de type `object` :

In [ ]:
numerical_selector = col_selector(dtype_exclude=object)
numerical_cols = numerical_selector(df)

Dans le cas où nous voudrions utiliser un vecteur *one-hot* pour les variables qualitatives et une *Z score normalization* pour les variables quantitatives, instancions une variable `preprocessor` pour établir les étapes de la transformation des données :

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

preprocessor = ColumnTransformer([
    ('one-hot-encoder', OneHotEncoder(handle_unknown="ignore"), categorical_cols),
    ('standard-scaler', StandardScaler(), numerical_cols)
])

La dernière phase est l’assemblage du pipeline :

In [ ]:
from sklearn.pipeline import make_pipeline

model = make_pipeline(preprocessor, LogisticRegression())
model